In [217]:
import re
import requests
import json
import pandas as pd
import sets
from collections import namedtuple

In [39]:
with open('data/test/sample-response.json','r') as f:
    bikejson = json.load(f)
    
type(bikejson)

bikejson.viewkeys()

dict_keys([u'type', u'features', u'metadata'])

In [170]:
payload = {'page': 10, 'include_submissions': 'false'} 
url = 'http://suggest.bayareabikeshare.com/api/places'

response = requests.get(url,payload)
text = response.text

ids = re.findall(r'id":\s(\d+)', text)
print len(ids)
# print ids
print len(sets.Set(ids))


supports = re.findall(r'support", "length":\s(\d+)', text)
# supports = re.findall(r'support...................', text)
print len(supports)
# print len(supports)

st_json = response.json()
print st_json.keys()
print st_json['features'][0].keys()
print st_json['features'][0]['properties'].keys()
print st_json['features'][0]['properties']['url']
st_json['features'][0]['properties']['submission_sets']['support']['length']
#intersting features uner properties: id, location, submission_sets, description, work, shopping
#under location: u'adminArea#, LatLng, postalCode

200
100
67
[u'type', u'features', u'metadata']
[u'geometry', u'type', u'properties', u'id']
[u'dataset', u'shopping', u'attachments', u'updated_datetime', u'url', u'submission_sets', u'visible', u'location_type', u'created_datetime', u'location', u'submitter', u'home', u'submitter_name', u'user_token', u'id', u'description']
http://shareaboutsapi2.herokuapp.com/api/v2/fkh/datasets/babs/places/356891


8

In [125]:
# save all jsons to disk
url = 'http://suggest.bayareabikeshare.com/api/places'
pages = range(1,48)

for page in pages: 
    payload = {'page': page, 'include_submissions': 'false'}
    response = requests.get(url,payload)
    st_json = response.json()
    with open('data/suggestion-json/page'+str(page),'w') as f:
        json.dump(st_json, f)
        


In [186]:
# function to parse out infomation from json, return a namedtuple
def parseJson(st_json):
    stationBasic = namedtuple('stationBasic', ['stid','lat','lng','weight'])
    stationBasics = []
    for i in range(len(st_json['features'])):
        stid = st_json['features'][i]['id']
        lat = st_json['features'][i]['geometry']['coordinates'][1]
        lng = st_json['features'][i]['geometry']['coordinates'][0]
        if st_json['features'][i]['properties']['submission_sets'].has_key('support') == True:
            weight = st_json['features'][i]['properties']['submission_sets']['support']['length']
        else:
            weight = 0
        stationBasics.append( stationBasic(stid = stid, lat = lat, lng = lng, weight = weight) )
    return stationBasics

test = st_json
testvalue = parseJson(test)
# test if has valid supports
# df = pd.DataFrame(result)
# df[3].value_counts()
# testvalue

In [216]:
# read jsons and parse out the needed data
pages = range(1,48)

parsed = []
for page in pages: 
    filename = 'data/suggestion-json/page' + str(page)
    with open(filename, 'r') as f:
        st_json = json.load(f)
#         print len(parseJson(st_json))
        parsed.extend( parseJson(st_json) )
        
columns = ['stid','lat','lng','weight']
df  = pd.DataFrame(parsed, columns=['stid','lat','lng','weight'])
df['weight'].sum()
df['lng'].describe()

df.to_csv('data/suggestions.csv',index = False)

In [222]:
df = pd.read_csv('data/suggestions.csv')
print df['weight'].sum()

30900
